In [1]:
import os
from pyspark.sql import SparkSession
# Create Spark session
spark = SparkSession.builder \
    .appName("KafkaSparkNotebook")\
    .config("spark.master", "spark://localhost:7077") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.3") \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/10/08 23:21:24 WARN Utils: Your hostname, AnshulLaptop resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/10/08 23:21:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/abheemreddy/spark/spark_env/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/abheemreddy/.ivy2/cache
The jars for the packages stored in: /home/abheemreddy/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-9a25e69a-d5ca-4bad-b046-0a7806a2ec15;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.3 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.3 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.5 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
downloading https://repo1.maven.org/maven2/org/apach

In [2]:
df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka1:9092,kafka2:9092") \
    .option("subscribe", "sample_topic") \
    .option("startingOffsets", "earliest") \
    .load()

In [4]:
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StructField, IntegerType
schema = StructType([
    StructField("number", IntegerType(), True)
])
# Deserialize JSON from Kafka
parsed_df = df.select(
    from_json(col("value").cast("string"), schema).alias("data")
).select("data.*")

In [7]:
# Print the deserialized data to the console
query = parsed_df \
    .writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

# Wait for the streaming query to terminate
query.awaitTermination()

24/10/08 23:30:56 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-af249dbf-05d8-4a71-9283-6ea3653359ba. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/10/08 23:30:56 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/10/08 23:31:06 WARN ClientUtils: Couldn't resolve server kafka1:9092 from bootstrap.servers as DNS resolution failed for kafka1
24/10/08 23:31:18 WARN ClientUtils: Couldn't resolve server kafka2:9092 from bootstrap.servers as DNS resolution failed for kafka2
24/10/08 23:31:18 WARN KafkaOffsetReaderAdmin: Error in attempt 1 getting Kafka offsets: 
org.apache.kafka.common.KafkaException: Failed to create new KafkaAdminClient
	at org.apache.kafka.clients.admin.KafkaA

StreamingQueryException: [STREAM_FAILED] Query [id = dd2c6f23-53bd-4d39-80e8-54e63a407759, runId = 198d2e19-2603-466c-845f-e8177e00f56b] terminated with exception: Failed to create new KafkaAdminClient